In [1]:
import konlpy
import pandas as pd
import numpy as np

데이터 파일을 읽어 리뷰 텍스트와 점수를 text_train, y_train 변수에 저장합니다. 데이터 파일의 내용은 번호, 텍스트, 레이블이 탭으로 구분되어 한 라인에 한개의 데이터 샘플이 들어 있습니다.

In [2]:
df_train = pd.read_csv('data-files/ratings_train.txt', 
                       delimiter='\t', 
                       keep_default_na=False)

In [3]:
df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  150000 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
text_train, y_train = df_train['document'].values, df_train['label'].values

같은 방식으로 테스트 데이터를 읽습니다.

In [5]:
df_test = pd.read_csv('data-files/ratings_test.txt', 
                      delimiter='\t', 
                      keep_default_na=False)
text_test = df_test['document'].values
y_test = df_test['label'].values

훈련 데이터와 테스트 데이터의 크기를 확인합니다.

In [6]:
len(text_train), np.unique(y_train, return_counts=True), np.bincount(y_train) # 이 정도면 괜찮다. 

(150000,
 (array([0, 1], dtype=int64), array([75173, 74827], dtype=int64)),
 array([75173, 74827], dtype=int64))

In [7]:
len(text_test), np.unique(y_test, return_counts=True), np.bincount(y_test)

(50000,
 (array([0, 1], dtype=int64), array([24827, 25173], dtype=int64)),
 array([24827, 25173], dtype=int64))

KoNLPy의 `Twitter` 클래스 개체를 만듭니다

In [8]:
#from konlpy.tag import Twitter
# twitter_tag = Twitter()
from konlpy.tag import Okt # Hannanum, Kkma
okt_tag = Okt()

`TfidfVectorizer` 의 토큰 파서를 대체하기 위해 `Twitter` 클래스를 사용하는 함수를 만듭니다.

In [19]:
def okt_tokenizer(text):
    word_list = okt_tag.morphs(text)
    #word_list에 대한 필요한 처리(예를 들면 불용어 제거, 관용어 제거 등) 필요
    return word_list

In [25]:
print(text_train[3])
okt_tokenizer(text_train[3])

교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정


['교도소', '이야기', '구먼', '..', '솔직히', '재미', '는', '없다', '..', '평점', '조정']

`min_df`, `ngram_range`, `C` 매개변수를 대상으로 그리드 서치를 수행합니다.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer # 문자를 숫자로 인코딩
from sklearn.linear_model import LogisticRegression # 위를 바탕으로 예측
from sklearn.pipeline import make_pipeline # 위의 두 가지를 일괄작업하여 관리 및 감독(CV,LR)
from sklearn.model_selection import GridSearchCV # 파이프라인 여러번 돌려 반복해라

twit_param_grid = {'tfidfvectorizer__min_df': [3, 5 ,7],
              'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
              'logisticregression__C': [0.1, 1, 10]}
# twit_pipe = make_pipeline(TfidfVectorizer(tokenizer=okt_tokenizer), LogisticRegression(solver="lbfgs", max_iter=1000))
twit_pipe = make_pipeline(TfidfVectorizer(tokenizer=okt_tokenizer), LogisticRegression(solver="liblinear")) # tokenizer로 분할
twit_grid = GridSearchCV(twit_pipe, twit_param_grid, cv=5) # 27 *5

# 그리드 서치를 수행합니다
twit_grid.fit(text_train[0:1000], y_train[0:1000]) # 1000개씩만 가지고 수행
print("최상의 크로스 밸리데이션 점수: {:.3f}".format(twit_grid.best_score_))
print("최적의 크로스 밸리데이션 파라미터: ", twit_grid.best_params_)

최상의 크로스 밸리데이션 점수: 0.717
최적의 크로스 밸리데이션 파라미터:  {'logisticregression__C': 1, 'tfidfvectorizer__min_df': 3, 'tfidfvectorizer__ngram_range': (1, 1)}


In [11]:
X_test_twit = twit_grid.best_estimator_.named_steps["tfidfvectorizer"].transform(text_test)
score = twit_grid.best_estimator_.named_steps["logisticregression"].score(X_test_twit, y_test)
print("테스트 세트 점수: {:.3f}".format(score))

테스트 세트 점수: 0.705


규제 파라미터의 범위를 확대하여 그리드 서치를 수행합니다.

In [12]:
mecab_param_grid = {'tfidfvectorizer__min_df': [3, 5 ,7],
              'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
              'logisticregression__C': [0.1, 1, 10, 100]}
mecab_pipe = make_pipeline(TfidfVectorizer(tokenizer=okt_tokenizer), LogisticRegression())
mecab_grid = GridSearchCV(mecab_pipe, mecab_param_grid, cv=5) 

# 그리드 서치를 수행합니다
mecab_grid.fit(text_train, y_train)
print("최상의 크로스 밸리데이션 점수: {:.3f}".format(mecab_grid.best_score_))
print("최적의 크로스 밸리데이션 파라미터: ", mecab_grid.best_params_)

KeyboardInterrupt: 

In [ ]:
X_test_mecab = mecab_grid.best_estimator_.named_steps["tfidfvectorizer"].transform(text_test)
score = mecab_grid.best_estimator_.named_steps["logisticregression"].score(X_test_mecab, y_test)
print("테스트 세트 점수: {:.3f}".format(score))